# Assignment 1 — Lars van den Heuvel  
### *Exploring Real-World Networks*

---

**Course:** Model Based Decisions (2025) — *MSc Complex Systems & Policy*  
**Lecturer:** Michael Lees  

---

**Datacollectors & Credits:**  
>J. Leskovec, D. Huttenlocher, J. Kleinberg. *Signed Networks in Social Media.* CHI 2010.  
>J. Leskovec, D. Huttenlocher, J. Kleinberg. *Predicting Positive and Negative Links in Online Social Networks.* WWW 2010.

>Link to website:https://snap.stanford.edu/data/wiki-Vote.html



In [31]:
import networkx as nx
G = nx.read_edgelist("C:/Users/lars_/MBDM/Week 2 - Networks/datasets/wiki-Vote.txt.gz", 
    comments='#', 
    create_using=nx.DiGraph(), 
    nodetype=int
)

In [32]:
nx.degree(G)
nx.shortest_path_length(G)
nx.clustering(G)

{30: 0.08068783068783068,
 1412: 0.024630541871921183,
 3352: 0.07534965521076632,
 5254: 0.04723590833069656,
 5543: 0.0881142366547144,
 7478: 0.05303392259913999,
 3: 0.12745098039215685,
 28: 0.055098887515451174,
 39: 0.1232905982905983,
 54: 0.1566265060240964,
 108: 0.08441558441558442,
 152: 0.10368482947863583,
 178: 0.056514418017259524,
 182: 0.06994266994266994,
 214: 0.034055829228243024,
 271: 0.05554646596858639,
 286: 0.20175438596491227,
 300: 0.1038961038961039,
 348: 0.07142857142857142,
 349: 0.11666666666666667,
 371: 0.05640012161751292,
 567: 0.11803713527851459,
 581: 0.1590909090909091,
 584: 0.22514619883040934,
 586: 0.1225296442687747,
 590: 0.1514762516046213,
 604: 0.1261904761904762,
 611: 0.09166666666666666,
 8283: 0.044444444444444446,
 25: 0.10074906367041199,
 6: 0.03712770692624775,
 8: 0.05986386528171231,
 19: 0.17552026286966046,
 23: 0.0901009657594381,
 29: 0.06375117189654221,
 33: 0.13264688067837674,
 35: 0.08370044052863436,
 50: 0.20397286

In [33]:
import pandas as pd
import networkx as nx

# Stel dat G je graph is
degree_dict = dict(G.degree())
clustering_dict = nx.clustering(G)

# Zet ze om in een DataFrame
df_metrics = pd.DataFrame({
    'degree': degree_dict,
    'clustering': clustering_dict
})

# Bekijk de eerste 5 rijen
print(df_metrics.head())

      degree  clustering
30        28    0.080688
1412      29    0.024631
3352     537    0.075350
5254     298    0.047236
5543     282    0.088114


In [14]:
#importing my dataset 
import pandas as pd
data_file = "C:/Users/lars_/MBDM/Week 2 - Networks/datasets/wiki-Vote.txt.gz"
df = pd.read_csv(data_file, comment='#', delim_whitespace=True, names=['source', 'target'])
print(df.head())

   source  target
0      30    1412
1      30    3352
2      30    5254
3      30    5543
4      30    7478


C:\Users\lars_\AppData\Local\Temp\ipykernel_10500\3904763776.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(data_file, comment='#', delim_whitespace=True, names=['source', 'target'])


In [15]:
#adding different labels to my dataframe
df = df.rename(columns={"source": "voter", "target": "candidate"})
print(df.head())


   voter  candidate
0     30       1412
1     30       3352
2     30       5254
3     30       5543
4     30       7478


In [28]:
n_voters = df["voter"].nunique()
n_candidates = df["candidate"].nunique()
print(n_voters)
print(n_candidates)

6110
2381


In [ ]:
votes_per_voter = df["voter"].value_counts()
print(votes_per_voter.head())



voter
2565    893
766     773
11      743
457     732
2688    618
Name: count, dtype: int64
